In [1]:
# When running on colab, run below commands
!mkdir dataset
%cd dataset
!wget https://cv.snu.ac.kr/research/VDSR/train_data.zip
!wget https://cv.snu.ac.kr/research/VDSR/test_data.zip
!unzip train_data.zip
!unzip test_data.zip
%cd ..

/content/dataset
--2018-12-03 18:00:35--  https://cv.snu.ac.kr/research/VDSR/train_data.zip
Resolving cv.snu.ac.kr (cv.snu.ac.kr)... 147.46.116.247
Connecting to cv.snu.ac.kr (cv.snu.ac.kr)|147.46.116.247|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 41628740 (40M) [application/zip]
Saving to: ‘train_data.zip’

train_data.zip      100%[===================>]  39.70M   846KB/s    in 39s     

2018-12-03 18:01:14 (1.02 MB/s) - ‘train_data.zip’ saved [41628740/41628740]

--2018-12-03 18:01:15--  https://cv.snu.ac.kr/research/VDSR/test_data.zip
Resolving cv.snu.ac.kr (cv.snu.ac.kr)... 147.46.116.247
Connecting to cv.snu.ac.kr (cv.snu.ac.kr)|147.46.116.247|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 62972123 (60M) [application/zip]
Saving to: ‘test_data.zip’

test_data.zip       100%[===================>]  60.05M   896KB/s    in 39s     

2018-12-03 18:01:54 (1.55 MB/s) - ‘test_data.zip’ saved [62972123/62972123]

Archive:  train_data

In [2]:
! pip install scikit-optimize

    100% |████████████████████████████████| 81kB 3.1MB/s 


In [3]:
#Importing the required packages
import random
import os
import cv2
from pathlib import Path
import glob
import string

import numpy as np
from keras.utils import Sequence
import tensorflow as tf
from matplotlib import pyplot as plt
from keras.models import Model
from keras.layers import Input, Add, PReLU, Conv2DTranspose, Concatenate, MaxPooling2D, UpSampling2D, Dropout
from keras.layers.convolutional import Conv2D
from keras.layers.normalization import BatchNormalization
from keras.callbacks import Callback, LearningRateScheduler, ModelCheckpoint, TensorBoard
from keras import backend as K
from keras.optimizers import *
from keras.utils import Sequence
# Using Keras Model in Scikit Learn
from keras.wrappers.scikit_learn import KerasRegressor
from skopt import BayesSearchCV

# include below until https://github.com/scikit-optimize/scikit-optimize/issues/718 is resolved
class BayesSearchCV(BayesSearchCV):
    def _run_search(self, x): raise BaseException('Use newer skopt')

Using TensorFlow backend.


In [0]:
def get_noise_model(noise_type="gaussian,0,50"):
    tokens = noise_type.split(sep=",")

    if tokens[0] == "gaussian":
        min_stddev = int(tokens[1])
        max_stddev = int(tokens[2])

        def gaussian_noise(img):
            noise_img = img.astype(np.float)
            stddev = np.random.uniform(min_stddev, max_stddev)
            noise = np.random.randn(*img.shape) * stddev
            noise_img += noise
            noise_img = np.clip(noise_img, 0, 255).astype(np.uint8)
            return noise_img

        return gaussian_noise

    elif tokens[0] == "clean":
        return lambda img: img

    elif tokens[0] == "text":

        min_occupancy = int(tokens[1])
        max_occupancy = int(tokens[2])

        def add_text(img):

            img = img.copy()
            h, w, _ = img.shape
            font = cv2.FONT_HERSHEY_SIMPLEX
            img_for_cnt = np.zeros((h, w), np.uint8)
            occupancy = np.random.uniform(min_occupancy, max_occupancy)

            while True:
                n = random.randint(5, 10)
                random_str = ''.join([random.choice(string.ascii_letters + string.digits) for i in range(n)])
                font_scale = np.random.uniform(0.5, 1)
                thickness = random.randint(1, 3)
                (fw, fh), baseline = cv2.getTextSize(random_str, font, font_scale, thickness)
                x = random.randint(0, max(0, w - 1 - fw))
                y = random.randint(fh, h - 1 - baseline)
                color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))
                cv2.putText(img, random_str, (x, y), font, font_scale, color, thickness)
                cv2.putText(img_for_cnt, random_str, (x, y), font, font_scale, 255, thickness)

                if (img_for_cnt > 0).sum() > h * w * occupancy / 100:
                    break
            return img

        return add_text


    elif tokens[0] == "impulse":

        min_occupancy = int(tokens[1])
        max_occupancy = int(tokens[2])

        def add_impulse_noise(img):
            occupancy = np.random.uniform(min_occupancy, max_occupancy)
            mask = np.random.binomial(size=img.shape, n=1, p=occupancy / 100)
            noise = np.random.randint(256, size=img.shape)
            img = img * (1 - mask) + noise * mask
            return img.astype(np.uint8)
        return add_impulse_noise

In [0]:
class NoisyImageGenerator(Sequence):
  
  def __init__(self, image_dir, source_noise_model, target_noise_model, batch_size=10000, image_size=128):

    self.image_paths = list(Path(image_dir).glob("*.jpg"))
    self.source_noise_model = source_noise_model
    self.target_noise_model = target_noise_model
    self.image_num = len(self.image_paths)
    self.batch_size = batch_size
    self.image_size = image_size

  def __len__(self):

    return self.image_num // self.batch_size

  def __getitem__(self):

    batch_size = self.batch_size
    image_size = self.image_size
    x = np.zeros((batch_size, image_size, image_size, 3), dtype=np.uint8)
    y = np.zeros((batch_size, image_size, image_size, 3), dtype=np.uint8)
    sample_id = 0

    while True:
      
      image_path = random.choice(self.image_paths)
      image = cv2.imread(str(image_path))
      h, w, _ = image.shape

      if h >= image_size and w >= image_size:
          h, w, _ = image.shape
          i = np.random.randint(h - image_size + 1)
          j = np.random.randint(w - image_size + 1)
          clean_patch = image[i:i + image_size, j:j + image_size]
          x[sample_id] = self.source_noise_model(clean_patch)
          y[sample_id] = self.target_noise_model(clean_patch)

          sample_id += 1

          if sample_id == batch_size:
              return x, y

In [0]:
class Generator(Sequence):
    def __init__(self, image_dir, val_noise_model):

        image_paths = list(Path(image_dir).glob("*.*"))
        self.image_num = len(image_paths)
        self.data = []

        for image_path in image_paths:
            y = cv2.imread(str(image_path))
            h, w, _ = y.shape
            y = y[:(h // 16) * 16, :(w // 16) * 16]  # for stride (maximum 16)
            x = val_noise_model(y)
            self.data.append(x)
            #print (x.shape)
            #print(y.shape)
            #self.data.append([np.expand_dims(x, axis=0), np.expand_dims(y, axis=0)])

    def __len__(self):
        return self.image_num

    def __getitem__(self, idx):
        return self.data[idx]
    
    def get_data(self):
        return self.data

In [0]:
source_noise_model = get_noise_model("text,0,50")
target_noise_model = get_noise_model( "text,0,0")
val_noise_model = get_noise_model("text,0,50")

image_dir = "dataset/291"
test_dir =  "dataset/Set14"

# batch_size = 4
# learning_rate = 0.003

# noisy_generator = NoisyImageGenerator(image_dir, source_noise_model, target_noise_model, batch_size=batch_size,
#                               image_size=64)
# val_generator = ValGenerator(test_dir, val_noise_model)
x, y = np.array(NoisyImageGenerator(image_dir, source_noise_model, target_noise_model).__getitem__())

In [0]:
def tf_log10(x):
    numerator = tf.log(x)
    denominator = tf.log(tf.constant(10, dtype=numerator.dtype))
    return numerator / denominator

def PSNR(y_true, y_pred):
    max_pixel = 255.0
    y_pred = K.clip(y_pred, 0.0, 255.0)
    return 10.0 * tf_log10((max_pixel ** 2) / (K.mean(K.square(y_pred - y_true))))

In [0]:
def get_unet_model(activation, learning_rate, optimizer, input_channel_num=3, out_ch=3, start_ch=64, depth=4, 
                   inc_rate=2., dropout=0.5, batchnorm=False, maxpool=True, upconv=True, residual=False):
    def _conv_block(m, dim, acti, bn, res, do=0):
        n = Conv2D(dim, 3, activation=acti, padding='same')(m)
        n = BatchNormalization()(n) if bn else n
        n = Dropout(do)(n) if do else n
        n = Conv2D(dim, 3, activation=acti, padding='same')(n)
        n = BatchNormalization()(n) if bn else n

        return Concatenate()([m, n]) if res else n

    def _level_block(m, dim, depth, inc, acti, do, bn, mp, up, res):
        if depth > 0:
            n = _conv_block(m, dim, acti, bn, res)
            m = MaxPooling2D()(n) if mp else Conv2D(dim, 3, strides=2, padding='same')(n)
            m = _level_block(m, int(inc * dim), depth - 1, inc, acti, do, bn, mp, up, res)
            if up:
                m = UpSampling2D()(m)
                m = Conv2D(dim, 2, activation=acti, padding='same')(m)
            else:
                m = Conv2DTranspose(dim, 3, strides=2, activation=acti, padding='same')(m)
            n = Concatenate()([n, m])
            m = _conv_block(n, dim, acti, bn, res)
        else:
            m = _conv_block(m, dim, acti, bn, res, do)

        return m

    i = Input(shape=(None, None, input_channel_num))
    o = _level_block(i, start_ch, depth, inc_rate, activation, dropout, batchnorm, maxpool, upconv, residual)
    o = Conv2D(out_ch, 1)(o)
    model = Model(inputs=i, outputs=o)
    
    model.compile(optimizer=optimizer(lr=learning_rate), loss="mae", metrics=[PSNR])
    
    return model

In [11]:
# log-uniform: understand as search over p = exp(x) by varying x
# this is our parameter grid
param_grid = {
    'batch_size': [4, 8, 16, 24],
    'learning_rate': (5e-4, 1e-2, 'log-uniform'),
    'activation' : ['relu', 'tanh', 'sigmoid'],
    'optimizer': [SGD, RMSprop, Adagrad, Adadelta, Adam]
}
# Wrap Keras model inside sci-kit learn
model = KerasRegressor(build_fn=get_unet_model, epochs=10)
# set up our optimiser to find the best params in 30 searches
bayes = BayesSearchCV(
    model,
    param_grid,
    random_state=1234,
    cv=2,
    verbose=10,
    return_train_score=True
)

bayes.fit(x, y)

print('Best params achieve a test score of', bayes.score(x, y), ':')
bayes.best_params_

Fitting 2 folds for each of 1 candidates, totalling 2 fits
[CV] activation=tanh, batch_size=24, learning_rate=0.0008723017716958026, optimizer=<class 'keras.optimizers.Adam'> 
Epoch 1/10
5000/5000 [==============================] - 207s 41ms/step - loss: 98.5787 - PSNR: 6.8264
Epoch 2/10
5000/5000 [==============================] - 191s 38ms/step - loss: 88.1741 - PSNR: 7.5703
Epoch 3/10
5000/5000 [==============================] - 191s 38ms/step - loss: 79.2287 - PSNR: 8.3058
Epoch 4/10
5000/5000 [==============================] - 191s 38ms/step - loss: 71.6733 - PSNR: 9.0143
Epoch 5/10
5000/5000 [==============================] - 191s 38ms/step - loss: 65.6230 - PSNR: 9.6842
Epoch 6/10
5000/5000 [==============================] - 191s 38ms/step - loss: 60.9898 - PSNR: 10.2680
Epoch 7/10
5000/5000 [==============================] - 191s 38ms/step - loss: 57.5652 - PSNR: 10.7716
Epoch 8/10
5000/5000 [==============================] - 191s 38ms/step - loss: 55.1039 - PSNR: 11.1758
Epoch

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 34.3min remaining:    0.0s


Epoch 1/10
5000/5000 [==============================] - 194s 39ms/step - loss: 98.4596 - PSNR: 6.8238
Epoch 2/10
5000/5000 [==============================] - 191s 38ms/step - loss: 88.1620 - PSNR: 7.5674
Epoch 3/10
5000/5000 [==============================] - 191s 38ms/step - loss: 79.2869 - PSNR: 8.2987
Epoch 4/10
5000/5000 [==============================] - 191s 38ms/step - loss: 71.8009 - PSNR: 9.0212
Epoch 5/10
5000/5000 [==============================] - 191s 38ms/step - loss: 65.8310 - PSNR: 9.6715
Epoch 6/10
5000/5000 [==============================] - 191s 38ms/step - loss: 61.2864 - PSNR: 10.2520
Epoch 7/10
5000/5000 [==============================] - 191s 38ms/step - loss: 57.9088 - PSNR: 10.7510
Epoch 8/10
5000/5000 [==============================] - 191s 38ms/step - loss: 55.4734 - PSNR: 11.1598
Epoch 9/10
5000/5000 [==============================] - 191s 38ms/step - loss: 53.7777 - PSNR: 11.4770
Epoch 10/10
5000/5000 [==============================] - 64s 13ms/step
[CV]  a

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 68.4min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 68.4min finished


Epoch 1/10
5000/5000 [==============================] - 250s 50ms/step - loss: 61.0871 - PSNR: 10.7537
Epoch 2/10
5000/5000 [==============================] - 247s 49ms/step - loss: 50.7501 - PSNR: 12.2661
Epoch 3/10
5000/5000 [==============================] - 247s 49ms/step - loss: 50.7477 - PSNR: 12.2768
Epoch 4/10
5000/5000 [==============================] - 247s 49ms/step - loss: 50.7568 - PSNR: 12.2831
Epoch 5/10
5000/5000 [==============================] - 247s 49ms/step - loss: 50.7646 - PSNR: 12.2769
Epoch 6/10
5000/5000 [==============================] - 248s 50ms/step - loss: 50.7559 - PSNR: 12.2797
Epoch 7/10
5000/5000 [==============================] - 248s 50ms/step - loss: 50.7588 - PSNR: 12.2745
Epoch 8/10
5000/5000 [==============================] - 248s 50ms/step - loss: 50.7640 - PSNR: 12.2775
Epoch 9/10
5000/5000 [==============================] - 248s 50ms/step - loss: 50.7601 - PSNR: 12.2776
Epoch 10/10
5000/5000 [==============================] - 83s 17ms/step
[C

...

Epoch 1/10
5000/5000 [==============================] - 197s 39ms/step - loss: 57.8485 - PSNR: 11.7034
Epoch 2/10
5000/5000 [==============================] - 192s 38ms/step - loss: 33.6241 - PSNR: 14.7601
Epoch 3/10
5000/5000 [==============================] - 192s 38ms/step - loss: 30.5135 - PSNR: 15.2938
Epoch 4/10
5000/5000 [==============================] - 192s 38ms/step - loss: 28.6611 - PSNR: 15.6190
Epoch 5/10
5000/5000 [==============================] - 192s 38ms/step - loss: 27.1123 - PSNR: 15.9355
Epoch 6/10
5000/5000 [==============================] - 193s 39ms/step - loss: 26.0566 - PSNR: 16.1459
Epoch 7/10
5000/5000 [==============================] - 193s 39ms/step - loss: 25.2901 - PSNR: 16.2891
Epoch 8/10
5000/5000 [==============================] - 193s 39ms/step - loss: 24.8017 - PSNR: 16.3695
Epoch 9/10
5000/5000 [==============================] - 193s 39ms/step - loss: 24.4076 - PSNR: 16.4430
Epoch 10/10
5000/5000 [==============================] - 64s 13ms/step
[C

[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 69.1min remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed: 69.1min finished


Epoch 1/10
5000/5000 [==============================] - 250s 50ms/step - loss: 98.6608 - PSNR: 6.8786
Epoch 2/10
5000/5000 [==============================] - 245s 49ms/step - loss: 92.8822 - PSNR: 7.2847
Epoch 3/10
5000/5000 [==============================] - 245s 49ms/step - loss: 89.2833 - PSNR: 7.5493
Epoch 4/10
5000/5000 [==============================] - 245s 49ms/step - loss: 86.4397 - PSNR: 7.7739
Epoch 5/10
5000/5000 [==============================] - 245s 49ms/step - loss: 84.1039 - PSNR: 7.9639
Epoch 6/10
5000/5000 [==============================] - 245s 49ms/step - loss: 82.0901 - PSNR: 8.1337
Epoch 7/10
5000/5000 [==============================] - 245s 49ms/step - loss: 80.3139 - PSNR: 8.2828
Epoch 8/10
5000/5000 [==============================] - 245s 49ms/step - loss: 78.7236 - PSNR: 8.4159
Epoch 9/10
5000/5000 [==============================] - 245s 49ms/step - loss: 77.2830 - PSNR: 8.5476
Epoch 10/10
5000/5000 [==============================] - 83s 17ms/step
[CV]  activ

[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed: 43.8min remaining:    0.0s


Epoch 1/10
5000/5000 [==============================] - 250s 50ms/step - loss: 100.7495 - PSNR: 6.7197
Epoch 2/10
5000/5000 [==============================] - 245s 49ms/step - loss: 95.6938 - PSNR: 7.0685
Epoch 3/10
5000/5000 [==============================] - 245s 49ms/step - loss: 92.6098 - PSNR: 7.2970
Epoch 4/10
5000/5000 [==============================] - 245s 49ms/step - loss: 90.0468 - PSNR: 7.4824
Epoch 5/10
5000/5000 [==============================] - 245s 49ms/step - loss: 87.9256 - PSNR: 7.6503
Epoch 6/10
5000/5000 [==============================] - 245s 49ms/step - loss: 86.0989 - PSNR: 7.7963
Epoch 7/10
5000/5000 [==============================] - 245s 49ms/step - loss: 84.4771 - PSNR: 7.9298
Epoch 8/10
5000/5000 [==============================] - 245s 49ms/step - loss: 83.0125 - PSNR: 8.0524
Epoch 9/10
5000/5000 [==============================] - 245s 49ms/step - loss: 81.6749 - PSNR: 8.1532
Epoch 10/10
 272/5000 [>.............................] - ETA: 1:18Buffered data w

In [0]:
# model = KerasRegressor(build_fn=get_unet_model, epochs=20, batch_size=8)
# model.fit(x=x, y=y)